In [1]:
import os

In [ ]:
%pwd
os.chdir(r"c:\Users\Admin\Desktop\text-summarizer\text-summarizer-project") 


In [19]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: str



In [20]:
pwd

'c:\\Users\\Admin\\Desktop\\text-summarizer\\text-summarizer-project'

In [21]:
import sys
sys.path.append(r"c:\Users\Admin\Desktop\text-summarizer\text-summarizer-project\src")


In [22]:
from text_summarizer.constants import *
from text_summarizer.utils.common import read_yaml, create_directories

In [23]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath: Path = CONFIG_FILE_PATH,
        params_filepath: Path = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])
        
        data_transformation_config = DataTransformationConfig(
            root_dir=Path(config.root_dir),
             data_path=Path(config.data_path),
            tokenizer_name=config.tokenizer_name
        )
        return data_transformation_config
    

In [24]:
import os
from  transformers import AutoTokenizer
from text_summarizer.logging import logger
from datasets import load_dataset, load_from_disk




In [30]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(self.config.tokenizer_name)

    def convert_examples_to_features(self, example_batch):
        inputs = self.tokenizer(
            example_batch['dialogue'],
            max_length=512,
            padding='max_length',
            truncation=True
        )
        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(
                example_batch['summary'],
                max_length=150,
                padding='max_length',
                truncation=True
            )
        return {
            'input_ids': inputs['input_ids'],
            'attention_mask': inputs['attention_mask'],
            'labels': target_encodings['input_ids']
        }
    
    def convert(self):
        dataset_samsum = load_from_disk(self.config.data_path)
        logger.info(f"Loaded dataset from {self.config.data_path}")
        dataset_samsum_pt= dataset_samsum.map(
            self.convert_examples_to_features,
            batched=True
        )
    
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir, "samsum_dataset"))

        

In [31]:
from datasets import load_from_disk
dataset = load_from_disk("artifacts/data_ingestion/samsum_dataset")
dataset


DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})

In [32]:
try:
    config_manager = ConfigurationManager()
    data_transformation_config = config_manager.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.convert()
except Exception as e:
    raise e

2025-11-27 02:12:23,997 - INFO - common - YAML file: config\config.yaml loaded successfully.
2025-11-27 02:12:24,002 - INFO - common - YAML file: params.yaml loaded successfully.
2025-11-27 02:12:24,004 - INFO - common - Directory created at: artifacts
2025-11-27 02:12:24,006 - INFO - common - Directory created at: artifacts/data_transformation


2025-11-27 02:12:25,915 - INFO - 2444299013 - Loaded dataset from artifacts\data_ingestion\samsum_dataset


Map:   0%|          | 0/14732 [00:00<?, ? examples/s]c:\Users\Admin\anaconda3\envs\textS\lib\site-packages\transformers\tokenization_utils_base.py:4114: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Saving the dataset (1/1 shards): 100%|██████████| 818/818 [00:00<00:00, 54461.97 examples/s]
